### Import python library

In [17]:
import numpy as np
import pandas as pd

### 1. Read data: augment_24group_1620.csv필요

In [18]:
data = pd.read_csv("bigcon_final_dataset.csv")

In [19]:
data.drop(["Unnamed: 0"],axis=1, inplace = True)

In [20]:
team = list(set(data.T_ID))

In [21]:
def shift_data(sel_col,df, shift_col):
    df_shift = df.loc[:,sel_col]
    
    for i in range(1,4):
        for c in shift_col:
            df_shift.loc[:,'shift_{}_{}'.format(c,i)] = df.loc[:,c].shift(i)
            
            
    df_shift.dropna(inplace = True)
    df_idx = list(df_shift.index)

    drop_idx = []
    for i in df_idx:
        idx_num = df_shift.loc[i,'IDX']
        if idx_num <= df_shift.loc[i,'shift_IDX_1'] or idx_num <= df_shift.loc[i,'shift_IDX_2'] or idx_num <= df_shift.loc[i,'shift_IDX_3'] :
            drop_idx.append(i)

    df_s_df = df_shift.drop(drop_idx).reset_index(drop = True)
    
    return df_s_df

### 2. Train 전처리

In [22]:
train = data[data["IDX"]!=777]

sel_col = ["T_ID","IDX","MERGE_IDX","PCT","ERA","AVG",'YEAR']
shift_col = sorted(list(set(list(train.columns))-set(['T_ID',"YEAR"])))

train_s_df = shift_data(sel_col, train, shift_col)
train_s_drop_df = train_s_df.drop(["IDX","MERGE_IDX","shift_IDX_1","shift_MERGE_IDX_1",
                                   "shift_IDX_2","shift_MERGE_IDX_2",
                                   "shift_IDX_3","shift_MERGE_IDX_3",
                                   "ERA","AVG","PCT"],axis = 1)

In [23]:
train_s_df.shape

(1000, 184)

### 3. Test 전처리

In [24]:
test = data[data["IDX"]==777]

In [25]:
tmp5 = data[(data["IDX"]==5)]
tmp4 = data[(data["IDX"]==4) & (data["MERGE_IDX"]==4)]
tmp3 = data[(data["IDX"]==3) & (data["MERGE_IDX"]==3)]

test = pd.concat([test,tmp5, tmp4, tmp3],axis=0)

test = test.sort_values(by=["YEAR","T_ID","IDX"]).reset_index(drop=True)

In [26]:
shift_col = sorted(list(set(list(test.columns))-set(['T_ID','YEAR'])))

test_s_df = shift_data(sel_col, test, shift_col)

test_s_df = test_s_df[test_s_df["IDX"]==777].reset_index(drop = True)
test_s_drop_df = test_s_df.drop(["IDX","MERGE_IDX","shift_IDX_1","shift_MERGE_IDX_1",
                                   "shift_IDX_2","shift_MERGE_IDX_2",
                                 "shift_IDX_3","shift_MERGE_IDX_3",
                                 "ERA","AVG","PCT"],axis = 1)

In [27]:
PCT_lstm_train_X = train_s_drop_df.reset_index(drop=True)
PCT_lstm_train_y =  train_s_df[["T_ID","YEAR","PCT"]]

PCT_lstm_test_X = test_s_drop_df.reset_index(drop=True)
PCT_lstm_test_y = test_s_df[["T_ID","YEAR","PCT"]]

In [12]:
PCT_lstm_train_X.to_csv("PCT_lstm_final_train_X33.csv",index=False)
PCT_lstm_train_y.to_csv("PCT_lstm_final_train_y33.csv",index=False)

PCT_lstm_test_X.to_csv("PCT_lstm_final_test_X33.csv",index=False)
PCT_lstm_test_y.to_csv("PCT_lstm_final_test_y33.csv",index=False)

### 4.  (서영님)split data  -> PCT/AVG/ERA dataset 가공

In [28]:
common_list = ['T_ID', 'YEAR', 'HEADER_NO','PCT']
hitter = ['AVG', 'OBP', 'SLG', 'OPS', 'RC', 'GPA', 'BA', 'SECA', 
          'TA', 'XR', 'ISO', 'wOBA', 'P_HRA_RT', 'DER']
pitcher = ['WHIP', 'LOB', 'ERA', 'FIP', 'H_9', 'K_9', 'BB_9', 
           'HR_9', 'oAVG', 'oSLG', 'oOBP', 'P_WHIP_RT', 'P2_WHIP_RT', 'CB_WHIP_RT','oISO', 'oOPS']

vs_hitter = ['vs_AVG', 'vs_OBP', 'vs_SLG', 'vs_OPS', 'vs_RC', 'vs_GPA', 'vs_BA', 'vs_SECA', 
             'vs_TA', 'vs_XR', 'vs_ISO', 'vs_wOBA', 'vs_P_HRA_RT', 'vs_DER']
vs_pitcher = ['vs_WHIP', 'vs_LOB', 'vs_ERA', 'vs_FIP', 'vs_H_9', 'vs_K_9', 'vs_BB_9', 
              'vs_HR_9', 'vs_oAVG', 'vs_oSLG', 'vs_oOBP', 'vs_P_WHIP_RT', 'vs_P2_WHIP_RT', 'vs_CB_WHIP_RT'
             ,'vs_oISO', 'vs_oOPS']

avg_col = common_list + hitter + vs_pitcher
era_col = common_list + pitcher + vs_hitter

In [30]:
train_col = list(train_s_drop_df.columns)
sel_avg_lst =train_col[:2]
sel_era_lst = train_col[:2]

for i in train_col[2:]:
    for k in avg_col:
        if k in i:
            sel_avg_lst.append(i)
            break
    
for i in train_col[2:]:
    for k in era_col:
        if k in i:
            sel_era_lst.append(i)
            break


In [33]:
# 타율
AVG_train_X = train_s_drop_df[sel_avg_lst]
AVG_train_y = train_s_df[["T_ID","YEAR","AVG"]]

AVG_test_X = test_s_drop_df[sel_avg_lst]
AVG_test_y = test_s_df[["T_ID","YEAR","AVG"]]

In [34]:
AVG_train_X.to_csv("AVG_lstm_final_train_X33.csv",index=False)
AVG_train_y.to_csv("AVG_lstm_final_train_y33.csv",index=False)

AVG_test_X.to_csv("AVG_lstm_final_test_X33.csv",index=False)
AVG_test_y.to_csv("AVG_lstm_final_test_y33.csv",index=False)

In [15]:
# 방어율
ERA_train_X = train_s_drop_df[sel_era_lst]
ERA_train_y = train_s_df[["T_ID","YEAR","ERA"]]

ERA_test_X = test_s_drop_df[sel_era_lst]
ERA_test_y = test_s_df[["T_ID","YEAR","ERA"]]

In [16]:
ERA_train_X.to_csv("ERA_lstm_final_train_X33.csv",index=False)
ERA_train_y.to_csv("ERA_lstm_final_train_y33.csv",index=False)

ERA_test_X.to_csv("ERA_lstm_final_test_X33.csv",index=False)
ERA_test_y.to_csv("ERA_lstm_final_test_y33.csv",index=False)

In [93]:
common_list = ['T_ID', 'YEAR', 'shift_HEADER_NO_1', 'shift_HEADER_NO_2',
               'shift_PF_1',  'shift_PF_2','shift_H_A_1','shift_H_A_2', 'shift_PCT_2','shift_PCT_2']

In [94]:
def add_shift(lst):
    res = []
    for i in lst:
        res.append('shift_'+i+'_1')
    for i in lst:
        res.append('shift_'+i+'_1')
    return res

In [95]:
# shift 추가

hitter = ['avg', 'OBP', 'SLG', 'OPS', 'RC', 'GPA', 'BA', 'SECA', 
          'TA', 'XR', 'ISO', 'wOBA', 'P_HRA_RT', 'DER']
pitcher = ['WHIP', 'LOB', 'era', 'FIP', 'H_9', 'K_9', 'BB_9', 
           'HR_9', 'oAVG', 'oSLG', 'oOBP', 'P_WHIP_RT', 'P2_WHIP_RT', 'CB_WHIP_RT']

vs_hitter = ['vs_avg', 'vs_OBP', 'vs_SLG', 'vs_OPS', 'vs_RC', 'vs_GPA', 'vs_BA', 'vs_SECA', 
             'vs_TA', 'vs_XR', 'vs_ISO', 'vs_wOBA', 'vs_P_HRA_RT', 'vs_DER']
vs_pitcher = ['vs_WHIP', 'vs_LOB', 'vs_era', 'vs_FIP', 'vs_H_9', 'vs_K_9', 'vs_BB_9', 
              'vs_HR_9', 'vs_oAVG', 'vs_oSLG', 'vs_oOBP', 'vs_P_WHIP_RT', 'vs_P2_WHIP_RT', 'vs_CB_WHIP_RT']

s_hit = add_shift(hitter)
s_pit = add_shift(pitcher)
s_vs_hit = add_shift(vs_hitter)
s_vs_pit = add_shift(vs_pitcher)

In [96]:
# 기존에서 제외된 column 목록
list(set(list(train_s_drop_df.columns))- set(s_hit+s_pit + s_vs_hit+ s_vs_pit + common_list))

['shift_vs_AVG_1',
 'shift_PCT_1',
 'shift_D_1',
 'shift_vs_PF_1',
 'shift_vs_AVG_2',
 'shift_L_1',
 'shift_WLS_1',
 'shift_vs_H_A_2',
 'shift_vs_ERA_2',
 'shift_AVG_1',
 'shift_AVG_2',
 'shift_vs_WLS_2',
 'shift_WLS_2',
 'shift_ERA_1',
 'shift_L_2',
 'shift_W_2',
 'shift_vs_ERA_1',
 'shift_W_1',
 'shift_vs_WLS_1',
 'shift_D_2',
 'shift_ERA_2',
 'shift_vs_PF_2',
 'shift_vs_H_A_1']

In [97]:
len(train_s_drop_df.columns), len(s_hit+s_pit + s_vs_hit+ s_vs_pit + common_list)

(144, 122)

### 5. 최종 데이터셋 생성

In [98]:
#승률
pct_extraction = common_list+ s_hit+ s_pit + s_vs_hit+ s_vs_pit

PCT_train_X = train_s_drop_df[pct_extraction]
PCT_train_y = train_s_df[["T_ID","YEAR","PCT"]]

PCT_test_X = test_s_drop_df[pct_extraction]
PCT_test_y = test_s_df[["T_ID","YEAR","PCT"]]

In [259]:
# 타율
avg_extraction = common_list + s_hit + s_vs_pit

AVG_train_X = train_s_drop_df[avg_extraction]
AVG_train_y = train_s_df[["T_ID","YEAR","avg"]]

AVG_test_X = test_s_drop_df[avg_extraction]
AVG_test_y = test_s_df[["T_ID","YEAR","avg"]]

In [260]:
# 방어율
era_extraction = common_list + s_pit + s_vs_hit

ERA_train_X = train_s_drop_df[era_extraction]
ERA_train_y = train_s_df[["T_ID","YEAR","era"]]

ERA_test_X = test_s_drop_df[era_extraction]
ERA_test_y = test_s_df[["T_ID","YEAR","era"]]

### 6. 저장

In [261]:
PCT_train_X.to_csv("PCT/PCT_train_X.csv",index=False)
PCT_train_y.to_csv("PCT/PCT_train_y.csv",index=False)

PCT_test_X.to_csv("PCT/PCT_test_X.csv",index=False)
PCT_test_y.to_csv("PCT/PCT_test_y.csv",index=False)

In [262]:
AVG_train_X.to_csv("AVG/AVG_train_X.csv",index=False)
AVG_train_y.to_csv("AVG/AVG_train_y.csv",index=False)

AVG_test_X.to_csv("AVG/AVG_test_X.csv",index=False)
AVG_test_y.to_csv("AVG/AVG_test_y.csv",index=False)

In [263]:
ERA_train_X.to_csv("ERA/ERA_train_X.csv",index=False)
ERA_train_y.to_csv("ERA/ERA_train_y.csv",index=False)

ERA_test_X.to_csv("ERA/ERA_test_X.csv",index=False)
ERA_test_y.to_csv("ERA/ERA_test_y.csv",index=False)

### <span style = "color:blue"> <참조> 팀 이름으로 one-hot 생성 </span>

In [267]:
#예시
PCT_train_X_oh = pd.concat([PCT_train_X, pd.get_dummies(PCT_train_X.T_ID)], axis = 1).drop(["T_ID"],axis=1)

In [268]:
PCT_train_X_oh

,YEAR,shift_HEADER_NO,shift_PF,shift_H_A,shift_PCT,shift_avg,shift_OBP,shift_SLG,shift_OPS,shift_RC,...,HH,HT,KT,LG,LT,NC,OB,SK,SS,WO
0,2016,0.0,972.833333,0.500000,0.500000,0.253317,0.325583,0.374167,0.699750,0.656958,...,0,0,0,1,0,0,0,0,0,0
1,2016,0.0,973.000000,0.500000,0.478261,0.291169,0.355625,0.403208,0.758833,0.825417,...,0,0,0,1,0,0,0,0,0,0
2,2016,0.0,968.625000,0.541667,0.375000,0.305842,0.363542,0.410750,0.774292,1.135625,...,0,0,0,1,0,0,0,0,0,0
3,2016,0.0,981.000000,0.416667,0.458333,0.297110,0.366333,0.432083,0.798417,1.201167,...,0,0,0,1,0,0,0,0,0,0
4,2016,0.0,972.833333,0.500000,0.500000,0.256379,0.327667,0.378792,0.706458,0.656500,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2915,2019,0.0,1025.375000,0.500000,0.500000,0.270784,0.324125,0.399000,0.723125,0.465083,...,0,0,0,0,0,0,0,0,1,0
2916,2019,0.0,1025.000000,0.375000,0.391304,0.269821,0.330208,0.387167,0.717375,0.432292,...,0,0,0,0,0,0,0,0,1,0
2917,2019,0.0,1024.416667,0.458333,0.416667,0.282396,0.333167,0.405708,0.738875,0.644417,...,0,0,0,0,0,0,0,0,1,0
2918,2019,0.0,1025.375000,0.500000,0.500000,0.271106,0.324375,0.391292,0.715667,0.469250,...,0,0,0,0,0,0,0,0,1,0


### +)

In [247]:
PCT_train_X.shape, PCT_train_y.shape, PCT_test_X.shape, PCT_test_y.shape

((2920, 62), (2920, 1), (50, 70), (50, 1))

In [248]:
PCT_train_y

,PCT
0,0.478261
1,0.375000
2,0.458333
3,0.583333
4,0.478261
...,...
2915,0.391304
2916,0.416667
2917,0.500000
2918,0.391304


In [249]:
AVG_train_X.shape, AVG_train_y.shape, AVG_test_X.shape, AVG_test_y.shape

((2920, 34), (2920, 1), (50, 34), (50, 1))

In [251]:
AVG_train_X

,T_ID,YEAR,shift_HEADER_NO,shift_PF,shift_H_A,shift_PCT,shift_avg,shift_OBP,shift_SLG,shift_OPS,...,shift_vs_H_9,shift_vs_K_9,shift_vs_BB_9,shift_vs_HR_9,shift_vs_oAVG,shift_vs_oSLG,shift_vs_oOBP,shift_vs_P_WHIP_RT,shift_vs_P2_WHIP_RT,shift_vs_CB_WHIP_RT
0,LG,2016,0.0,972.833333,0.500000,0.500000,0.253317,0.325583,0.374167,0.699750,...,8.705208,7.958750,4.053000,1.132250,0.249250,0.379667,0.312583,1.419500,1.544792,1.791750
1,LG,2016,0.0,973.000000,0.500000,0.478261,0.291169,0.355625,0.403208,0.758833,...,10.303583,6.593083,3.439667,0.589875,0.288875,0.407500,0.339292,1.796958,1.626833,1.971792
2,LG,2016,0.0,968.625000,0.541667,0.375000,0.305842,0.363542,0.410750,0.774292,...,11.237125,6.863167,3.305583,0.725458,0.307333,0.420042,0.350417,1.620750,1.630458,2.194542
3,LG,2016,0.0,981.000000,0.416667,0.458333,0.297110,0.366333,0.432083,0.798417,...,10.662417,6.665708,4.143000,0.917625,0.296250,0.438542,0.353000,1.930458,2.127708,2.208417
4,LG,2016,0.0,972.833333,0.500000,0.500000,0.256379,0.327667,0.378792,0.706458,...,8.809208,7.996583,4.041208,1.139333,0.251042,0.383000,0.314667,1.446292,1.560833,1.810500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2915,SS,2019,0.0,1025.375000,0.500000,0.500000,0.270784,0.324125,0.399000,0.723125,...,9.691583,7.013208,2.978167,0.913042,0.276917,0.411333,0.327042,1.454375,1.688542,1.381833
2916,SS,2019,0.0,1025.000000,0.375000,0.391304,0.269821,0.330208,0.387167,0.717375,...,8.919083,6.224750,3.072125,0.911458,0.255375,0.393042,0.298083,1.224458,1.956542,1.582417
2917,SS,2019,0.0,1024.416667,0.458333,0.416667,0.282396,0.333167,0.405708,0.738875,...,9.513167,6.590333,3.328208,0.700000,0.265417,0.387583,0.311542,1.435917,1.819250,1.563792
2918,SS,2019,0.0,1025.375000,0.500000,0.500000,0.271106,0.324375,0.391292,0.715667,...,9.691583,7.060083,2.978167,0.959917,0.276417,0.416458,0.327042,1.506458,1.633875,1.473500


In [253]:
ERA_train_X.shape, ERA_train_y.shape, ERA_test_X.shape, ERA_test_y.shape

((2920, 34), (2920, 1), (50, 34), (50, 1))

In [254]:
ERA_train_X

,T_ID,YEAR,shift_HEADER_NO,shift_PF,shift_H_A,shift_PCT,shift_WHIP,shift_LOB,shift_era,shift_FIP,...,shift_vs_RC,shift_vs_GPA,shift_vs_BA,shift_vs_SECA,shift_vs_TA,shift_vs_XR,shift_vs_ISO,shift_vs_wOBA,shift_vs_P_HRA_RT,shift_vs_DER
0,LG,2016,0.0,972.833333,0.500000,0.500000,1.595250,0.687917,5.358453,20.730708,...,0.870292,0.266833,0.289208,0.312042,0.767708,5.260250,0.125750,0.389000,0.303000,0.704083
1,LG,2016,0.0,973.000000,0.500000,0.478261,1.521333,0.711250,4.715603,18.914375,...,0.845500,0.249500,0.268792,0.296458,0.715333,4.609000,0.098375,0.372792,0.324333,0.669542
2,LG,2016,0.0,968.625000,0.541667,0.375000,1.563292,0.668500,5.584768,27.979500,...,0.994750,0.272500,0.285500,0.344042,0.808750,5.693083,0.153917,0.403250,0.296500,0.647625
3,LG,2016,0.0,981.000000,0.416667,0.458333,1.587875,0.658375,5.551402,27.325833,...,1.168750,0.274167,0.280625,0.379125,0.842542,5.810750,0.158125,0.413250,0.261625,0.667042
4,LG,2016,0.0,972.833333,0.500000,0.500000,1.631375,0.685667,5.492227,20.891833,...,0.927500,0.270792,0.295000,0.311250,0.780375,5.378500,0.125833,0.394417,0.306917,0.699417
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2915,SS,2019,0.0,1025.375000,0.500000,0.500000,1.421542,0.775458,3.964445,22.081708,...,0.484292,0.238917,0.256583,0.302708,0.676125,4.239500,0.111417,0.353000,0.203625,0.676042
2916,SS,2019,0.0,1025.000000,0.375000,0.391304,1.521917,0.685292,4.909091,19.320583,...,0.553375,0.236458,0.243208,0.326750,0.688167,4.435583,0.127375,0.355083,0.164500,0.720292
2917,SS,2019,0.0,1024.416667,0.458333,0.416667,1.379667,0.706583,4.478873,21.147583,...,1.085792,0.268708,0.293042,0.341458,0.831667,5.488583,0.134833,0.398917,0.304458,0.698083
2918,SS,2019,0.0,1025.375000,0.500000,0.500000,1.449292,0.765542,4.047907,22.086333,...,0.372667,0.232583,0.247625,0.291542,0.643458,4.027083,0.108167,0.343625,0.194042,0.676958
